In [1]:
import pandas as pd
import feedparser
from newspaper import Article
import re
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
def get_contents(link):
        article = Article(link, language='ko')
        article.download()
        article.parse()
        return article.text

def cleaning_korean(content):
    content = content.replace('\n\n','')
    content = ' '.join(content.split(' '))

    #클리닝1
    content = re.sub(r'... 기자', '', content)
    content = re.sub(r'...기자', '', content)

    #괄호안 제거 
    ''' 괄호안 내용 제거하기 '''
    pattern_bracket1 = r'\([^)]*\)'
    pattern_bracket2 = r'\{[^)]*\}'
    pattern_bracket3 = r'\<[^)]*\>'
    pattern_bracket4 = r'\[[^)]*\]'

    x = '이건 {괄호 안의 불필요한 정보를} 삭제하는 코드다.'
    content = re.sub(pattern=pattern_bracket1, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket2, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket3, repl='', string= content)
    content = ' '.join(content.split())
    content = re.sub(pattern=pattern_bracket4, repl='', string= content)
    content = ' '.join(content.split())
    #클리닝
    content = content.lower() #lower case
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    content = re.sub(pattern=pattern, repl='', string=content)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    content = re.sub(pattern=pattern, repl='', string=content)
    # 특수기호 특수
    content = re.sub('[-=+#/\?:^$@*\"※~%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》Δ△·“=◎<>▷]', '', content)
    content = re.sub('[一-龥]', '',content)
    #pattern = '[^\w\s]'         # 특수기호제거
    #content = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\:\;\!\-\,\_\~\$\'\"]', '',str(content)) #remove punctuation
    content = re.sub(r"[^a-zA-Z0-9가-힣\.\s]","",content)
    content = re.sub(pattern=pattern, repl='', string=content)
    content = re.sub(r'\s+', ' ', content) #remove extra space
    content = re.sub(r'<[^>]+>','',content) #remove Html tags
    content = re.sub(r'\s+', ' ', content) #remove spaces
    content = re.sub(r"^\s+", '', content) #remove space from start
    content = re.sub(r'\s+$', '', content) #remove space from the end
    return content

In [3]:
dflink = pd.read_csv('./news_url.csv')

In [4]:
p_contents = []
for url in dflink['p']:
    try:
        p_contents.append(  cleaning_korean(get_contents(url) ))
        print(url)
    except Exception as e:
        print(e)
        
print('----------')
n_contents = []
for url in dflink['n']:
    try:
        n_contents.append(  cleaning_korean(get_contents(url) ))
        print(url)
    except Exception as e:
        print(e)
    

https://www.thebell.co.kr/free/content/ArticleView.asp?key=202201211614241520101893&lcode=00
https://www.ajunews.com/view/20211201050234882 
https://www.fnnews.com/news/202111281735005130 
Article `download()` failed with 404 Client Error: Not Found for url: https://www.hankyung.com/economy/article/202212145008i%20 on URL https://www.hankyung.com/economy/article/202212145008i 
https://www.ajunews.com/view/20221130171222768
https://www.edaily.co.kr/news/read?newsId=02079526592972528&mediaCodeNo=257
https://www.etoday.co.kr/news/view/387113
https://www.newspim.com/news/view/20110411000013
https://www.thebell.co.kr/free/content/ArticleView.asp?key=201801040100006950000414&lcode=00 
Article `download()` failed with 404 Client Error: Not Found for url: https://www.hankyung.com/economy/article/2018010327901%20 on URL https://www.hankyung.com/economy/article/2018010327901 
http://m.ddaily.co.kr/m/m_article/?no=164321 
https://www.edaily.co.kr/news/read?newsId=03440726619074112&mediaCodeNo=2 


In [5]:
from konlpy.tag import Mecab


def content2noun_mecab(series_content, save_path = None):
    try:
        tokenizer = Mecab() 
    except:
        tokenizer = Mecab('C:\mecab\mecab-ko-dic') 
    res_list = []
    print('------ start mecab tokenizer ------')
    for row in series_content:
        try:
            res = ' '.join(tokenizer.nouns(row))
            res_list.append(res)
        except Exception as e:
            print(e)
            res_list.append('')
    return res_list

In [6]:
pc = []
for c in p_contents:
    pc.append(content2noun_mecab(c))

nc =[]
for c in n_contents:
    nc.append(content2noun_mecab(c))

------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------
------ start mecab tokenizer ------


In [7]:
pres = content2noun_mecab(p_contents)
nres = content2noun_mecab(n_contents)

------ start mecab tokenizer ------
------ start mecab tokenizer ------


In [8]:
pl = []
for res in pres:
    pl = pl+res.split(' ')
    
nl = []
for res in nres:
    nl = nl+res.split(' ')

In [9]:
pclean = []
for cc in pl:
    if len(cc) < 2:
        continue
    else:
        pclean.append(cc)
        
nclean = []
for cc in nl:
    if len(cc) < 2:
        continue
    else:
        nclean.append(cc)

---

In [10]:
from wordcloud import WordCloud

In [14]:
def get_count(pclean):
    dfp = pd.DataFrame()
    dfp['count'] = pclean
    
    dfpc = dfp.value_counts().to_frame()
    dfpc['word'] =  dfpc.index
    dfpc = dfpc.reset_index(drop=True)
    dfpc['word'] = dfpc['word'].apply(lambda x : x[0])
    dfpc.columns = ['count','word']
    
    #dfpc = dfpc.loc[dfpc['count'] > 1]

    return dfpc

In [15]:
dfpc = get_count(pclean)
dfnc = get_count(nclean)

In [16]:
dfpc.to_csv('ppp_raw_include1.csv')
dfnc.to_csv('nnn_raw_include1.csv')

---

In [17]:
import numpy as np

In [18]:
dfpc = pd.read_csv('./10개미만제거/ppp.csv')
mx = dfpc['count'].max()
mn = dfpc['count'].min()
#dfpc['count'] = dfpc['count'].apply(lambda x : int(  (mx - x)/(mx - mn) * 100 ))


if len(dfpc) !=  len(set(dfpc['word'].values)):
    print('dif 12312313414124')
pdict = dict(zip(dfpc['word'] , dfpc['count']))
wc = WordCloud(font_path='malgun', width=800, height=800, scale=1.0, max_font_size=800 ,  background_color = 'white')
gen = wc.generate_from_frequencies(pdict)
wc.to_file('pp_new.png')
#plt.figure()
#plt.imshow(gen)
#Counter(list(dfpc['word'].values))

dif 12312313414124


In [19]:
dfpc = pd.read_csv('./10개미만제거/nnn.csv', index_col = 0)
mx = dfpc['count'].max()
mn = dfpc['count'].min()
#dfpc['count'] = dfpc['count'].apply(lambda x : int( np.log2( (mx - x)/(mx - mn) + 0.0001 ) + 10 ))

if len(dfpc) !=  len(set(dfpc['word'].values)):
    print('dif 12312313414124')
pdict = dict(zip(dfpc['word'] , dfpc['count']))
wc = WordCloud(font_path='malgun', width=800, height=800, scale=1.0, max_font_size=800, background_color = 'white')
gen = wc.generate_from_frequencies(pdict)
wc.to_file('nn_new.png')
#plt.figure()
#plt.imshow(gen)
#Counter(list(dfpc['word'].values))

dif 12312313414124


In [20]:
import numpy as np
from scipy.optimize import minimize

xs = np.random.randn(1000 , 20)
ys = np.random.randn(1000 , 1) * 20

# Define the objective function to be minimized
def objective(x):
    return x[0]**2 + x[1]**2

# Define the stochastic gradient of the objective function
def stochastic_gradient(x):
    return np.array( [np.random.normal(2*x[0]), np.random.normal(2*x[1])])

# Define the bounds of the search space
bounds = [(-1, 1), (-1, 1)] * 10

# Call the minimize function with the stochastic gradient method
result = minimize(objective, x0=[0, 0], method='SLSQP', jac=stochastic_gradient, bounds=bounds)

# Print the optimization result
print(result)

     fun: 3.6781201935056934e-08
     jac: array([ 0.57693767, -0.11748221])
 message: 'Optimization terminated successfully'
    nfev: 12
     nit: 1
    njev: 1
  status: 0
 success: True
       x: array([-1.87927581e-04,  3.82678224e-05])


In [21]:
x = [10,20]
np.array( [np.random.normal(2*x[0]), np.random.normal(2*x[1])])

array([20.15358615, 40.07703828])